In [1]:
import minio

In [ ]:
print("hello world!")

### Spark сессия

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

drivers = [
    "/home/jovyan/work/spark-jars/hadoop-aws-3.3.4.jar",             # S3
    "/home/jovyan/work/spark-jars/aws-java-sdk-bundle-1.12.262.jar", # S3
    "/home/jovyan/work/spark-jars/wildfly-openssl-1.0.7.Final.jar",  # S3
    "/home/jovyan/work/spark-jars/postgresql-42.6.0.jar",            # PostgreSQL
]

spark = (SparkSession.builder
         .appName("mustdayker-Spark2")
         .master("spark://spark-master:7077") 
         .config("spark.jars", ",".join(drivers))
         .getOrCreate()
        )

25/11/04 15:42:37 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


### Читаем датафрейм

In [2]:
df_2023_01 = spark.read.parquet("s3a://silver/nyc-taxi-data/yellow_tripdata_2023-01")

25/11/04 15:42:42 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties
                                                                                

In [ ]:
df_2024_01 = spark.read.parquet("s3a://bronze/nyc-taxi-data/yellow_tripdata_2024-01.parquet")
df_2025_01 = spark.read.parquet("s3a://bronze/nyc-taxi-data/yellow_tripdata_2025-01.parquet")

In [ ]:
layer = "silver"

df = (spark.read
      .format("parquet")
      .option("mergeSchema", "true")
      .load([
          f"s3a://{layer}/nyc-taxi-data/yellow_tripdata_2023-*",
          f"s3a://{layer}/nyc-taxi-data/yellow_tripdata_2024-*",
          f"s3a://{layer}/nyc-taxi-data/yellow_tripdata_2025-*",
      ])
     )

In [3]:
print(f"Количетсво строк: {df_2023_01.count()}\n")

[Stage 1:==================================>                       (6 + 4) / 10]

Количетсво строк: 3066766



In [ ]:
df.show(1)

In [ ]:
print(f"Количетсво строк: {df.count()}\n")

(df.withColumn("year",  F.year("tpep_pickup_datetime"))
   .withColumn("month", F.month("tpep_pickup_datetime"))
   .select("year", 
           "month", 
           "VendorID",
           "tpep_pickup_datetime", 
           "passenger_count", 
           "trip_distance",
           "airport_fee",
          )
   .groupBy("year", "month")
   .agg(
       F.count("*").alias("total_records"),
       F.sum("airport_fee").alias("Обдираловка"),
       )
   .orderBy("year", "month")
   .show(50)
)

### Исследуем

In [ ]:
from pyspark.sql import functions as F
from pyspark.sql.types import DoubleType, IntegerType

def standardize_nyc_taxi_data(input_path, output_path):
    """
    Стандартизирует данные NYC Taxi:
    - Приводит все названия колонок к нижнему регистру
    - Приводит типы данных к унифицированным
    - Сохраняет в указанный путь
    """

    output_path = output_path.replace('.parquet', '')
    
    # Читаем исходные данные
    df = spark.read.parquet(input_path)
    
    # 1. Приводим все названия колонок к нижнему регистру
    for col_name in df.columns:
        df = df.withColumnRenamed(col_name, col_name.lower())
    
    # 2. Определяем маппинг типов для унификации
    type_mapping = {
        # Числовые идентификаторы - в Integer
        "vendorid": IntegerType(),
        "pulocationid": IntegerType(), 
        "dolocationid": IntegerType(),
        "payment_type": IntegerType(),
        "ratecodeid": IntegerType(),
        
        # Пассажиры - в Integer (более логично)
        "passenger_count": IntegerType(),
        
        # Денежные суммы - в Double (для точности)
        "fare_amount": DoubleType(),
        "extra": DoubleType(),
        "mta_tax": DoubleType(),
        "tip_amount": DoubleType(),
        "tolls_amount": DoubleType(),
        "improvement_surcharge": DoubleType(),
        "total_amount": DoubleType(),
        "congestion_surcharge": DoubleType(),
        "airport_fee": DoubleType(),
        "cbd_congestion_fee": DoubleType(),
        
        # Дистанция - в Double
        "trip_distance": DoubleType()
    }
    
    # 3. Применяем преобразование типов с обработкой ошибок
    for col_name, target_type in type_mapping.items():
        if col_name in df.columns:
            df = df.withColumn(
                col_name, 
                F.coalesce(
                    F.col(col_name).cast(target_type), 
                    F.lit(0 if target_type == IntegerType() else 0.0)
                )
            )
    
    # 4. Гарантируем порядок колонок для consistency
    expected_columns = [
        "vendorid", "tpep_pickup_datetime", "tpep_dropoff_datetime",
        "passenger_count", "trip_distance", "ratecodeid", "store_and_fwd_flag",
        "pulocationid", "dolocationid", "payment_type", "fare_amount", "extra",
        "mta_tax", "tip_amount", "tolls_amount", "improvement_surcharge",
        "total_amount", "congestion_surcharge", "airport_fee", "cbd_congestion_fee"
    ]
    
    # Выбираем только существующие колонки в правильном порядке
    final_columns = [col for col in expected_columns if col in df.columns]
    df_standardized = df.select(final_columns)
    
    # 5. Сохраняем с оптимальными настройками
    (df_standardized
     .coalesce(1)
     .write
     .mode("overwrite")
     .option("compression", "snappy")  # хороший баланс скорость/сжатие
     .parquet(output_path)
    )
    
    print(f"✅ Стандартизировано: {input_path} -> {output_path}")
    # print(f"📊 Схема после стандартизации:")
    # df_standardized.printSchema()
    
    return df_standardized

In [ ]:
# Пример использования для одного файла
input_path = "s3a://bronze/nyc-taxi-data/yellow_tripdata_2023-01.parquet"
output_path = "s3a://silver/nyc-taxi-data/yellow_tripdata_2023-01.parquet"

standardized_df = standardize_nyc_taxi_data(input_path, output_path)

In [ ]:
def process_all_nyc_taxi_files():
    """Обрабатывает все файлы NYC Taxi из Bronze в Silver"""
    
    # Получаем список всех parquet файлов в bronze
    bronze_files = spark.sql(f"""
        SELECT path 
        FROM (
            SELECT input_file_name() as path 
            FROM parquet.`s3a://bronze/nyc-taxi-data/`
        ) 
        GROUP BY path
    """).collect()
    
    print(f"📁 Найдено файлов для обработки: {len(bronze_files)}")
    
    for i, row in enumerate(bronze_files, 1):
        input_path = row['path']
        # Извлекаем имя файла из пути
        file_name = input_path.split('/')[-1]
        output_path = f"s3a://silver/nyc-taxi-data/{file_name}"
        
        print(f"🔄 Обрабатываю ({i}/{len(bronze_files)}): {file_name}")
        
        try:
            standardize_nyc_taxi_data(input_path, output_path)
        except Exception as e:
            print(f"❌ Ошибка при обработке {file_name}: {e}")
    
    print("🎉 Все файлы обработаны!")

In [ ]:
import time

start_time = time.time()

# Запускаем обработку всех файлов
process_all_nyc_taxi_files()

end_time = time.time()
execution_time = end_time - start_time
print(f"Время выполнения: {execution_time:.4f} секунд")